## Using NexMotion API to Control MiniBot
### Pre-requirement:
* Python2.7 32bit version (NexMotion.dll is 32bit version)
* Create MiniBot Configuration from NexMotion Studio (NexMotionNcf.ncf)
* For controlling real MiniBOT, do homing process in NexMotion Studio first.

### How to run this demo
* Select each cell by left click it (such as click on `[1]` ) and press `shift+Enter` to run the cell. Also, could press direction key (up/down) to select cells.
* (Option) You cloud left click inside area of each cell if you want to change scripts and test.
* (Option) In JupyterLab mode, Right Click it and select `Create New View for Output`. 

#### 1. Include Necessary Libarary with environment variable

In [1]:
%config IPCompleter.greedy=True
# include nexmotion library
import nexmotion as nm
from nexmotion.errors import *
from nexmotion.constants import *
import numpy as np
import pandas as pd
dll_path = None

#### 2. Start Device
Note: It will take few second to load NexMotion.dll. the cell indicator will change from `[*]` to `[number]` when finish.

In [2]:
if dll_path is None:
    nmc = nm.Control()  # will use default path for dll and ini file
else:
    nmc = nm.Control(dll_path=dll_path)
ret = nmc.deviceOpenup(DEVICE_TYPE_SIMULATOR)  # use DEVICE_TYPE_ETHERCAT to connect real Minibot
if ret != nm.errors.SUCCESS:
    print( "NMC_DeviceOpenUp error, error code =", ret)

Dynamics library version = 10121027 ( 1 , 1 , 2 , 1027 )
device id = 0


#### 3. Check Device State
Get the Device State.
Here are the 4 possible state. If nothing wrong, the device state will be **4**

| Name   |      Value      |  Description |
|--------|:-------------:|------:|
| NMC_DEVICE_STATE_INIT      | 1 | Initial |
| NMC_DEVICE_STATE_READY     | 2 | Ready |
| NMC_DEVICE_STATE_ERROR     | 3 | Error |
| NMC_DEVICE_STATE_OPERATION | 4 | Operation |

In [3]:
ret = nmc.deviceGetState()
if ret != SUCCESS:
    print( "NMC_DeviceGetState error, error code =", ret)
else:
    if nmc.devState_.value != DEVICE_STATE_OPERATION:
        print( "device state is not OPERATION" )
    else:
        print( "device state =", nmc.devState_.value)


device state = 4


#### 4. Display Virtual Robot for visualization

In [4]:
# It Will show up on the desktop, not Jupyter (also not work on JupyterHub demo).
if nm.api.HAS_ZUGBRUECKE is None:
    nmc.group3DShow(top_=True)
    nmc.group3DDrawPath(enable=True)
    print("The NexMotion built-in 3D Robot View will show up on the pc desktop, no Jupyter.")
else:
    print("The NexMotion built-in 3D Robot View will not show on the JupyterHub (MyBinder).")
    print("Use following maplotlib-based class to create a 3D tf View.")

The NexMotion built-in 3D Robot View will not show on the JupyterHub (MyBinder).
Use following maplotlib-based class to create a 3D tf View.


### Testing Robot 3D tf View
This cell show how to use MplVisual Class, matplotlib-base, to show transformation coordination of each joint in 3D (like ROS-RViz tf view).

#### Hint: 
In JupyterLab mode, Right Click it (left `[#]`) and select `Create New View for Output`. JupyterLab will put 3D tf view on another tab. Make it easy to check when running the rest of cell.

Note: Currently, the tf view will not automatically update when you move robot joints. you need to run `nmv.update_tf_view(jnt_pos)` manually.

In [5]:
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

# Create figure canvas to hold 3D tf view
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Initial MplVisual class with built-in dh parameter
nmv = nm.api.MplVisual(ax)   

# Compute tf matrix with initial joint value (0, 90, 0, 0, -90, 0)
nmv.get_tf_data()
nmv.draw_tf_view()  # Draw 3D tf view by Matplotlib
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
nmv.update_tf_view([0, 45, 0, 0, -45, 90])

#### 5. Get device information: Count of group

In [7]:
nmc.deviceGetGroupCount()
print( "count of group index", nmc.numGroup_.value)

count of group index 1


#### 6. Get device information: Count of Group Axis
For MiniBot, the below code will output: 6

In [8]:
nmc.deviceGetGroupAxisCount()
print( "group index", nmc.index_.value, ": count of group axis = ", nmc.numGroupAxis_.value)

group index 0 : count of group axis =  6


#### 7. Clean alarm of drives of each group
When sending movement command (such as moveLine, groupLine), the robot may not execute correctly (such as desired position is outside its workspace or in singular point. Then, the controller will set group state to `GROUP_ERROR_STOP`).
These two are useful command to clean alarm or reset state of robot.

In [9]:
nmc.groupResetDriveAlmAll()
nmc.groupResetState()

0

## 8. Check group state
Here are all possible group (movement) state:
Get the state of group. Please refer to the below table for details.

|Value | State | Description | 
|------|:-------:|-------------:| 
| 0 | GROUP_DISABLE     | A group axis is disabled. | 
| 1 | GROUP_STAND_STILL | All group axes are enabled. | 
| 2 | GROUP_STOPPED     | After NMC_GroupStop() is called, the group is stopped. |
| 3 | GROUP_STOPPING    | After NMC_GroupStop() is called, the group is stopping. |
| 4 | GROUP_MOVING      | The group is moving. |
| 5 | GROUP_HOMING      | The group is homing. |
| 6 | GROUP_ERROR_STOP  | An error is occurred in a group axis. |

##### Note: if Group State == 6 (error stop), please run step 7 to reset state.
##### Note: if Group State == 0 (disable), please run step 9 to servo on in order to move robot arm.


In [11]:
ret = nmc.groupGetState()
if ret != SUCCESS:
    print( "groupGetState error, error code =", ret)
else:
    print( "Group State = ", nmc.groupState_.value)

Group State =  1


#### 9. Enable all axes of group (Servo On)
MiniBot will make a sound when it is enabled.
And, group sate will be changed (could run step **8** to check again)  
Note: need to Enable before running any movement command

In [10]:
ret = nmc.groupEnable()
if ret != SUCCESS:
    print( "NMC_DeviceEnableAll error, error code =", ret)
else:
    print( "enable all done" )

enable all done


### 10. Show Actual Joint Position
This step can be called to check actual joint values after you move or home the Minibot.  
For controlling real MiniBOT, make sure you do homing process (or this command will not show correctly)

#### Note:
After run below cell, click `Run Interact` to get current joint values and update the tf view. 
You could right click and select `Create New View for Output` and move `Run Interact` button wiht the tf view. It is quick to get join value and update the tf view. 

In [12]:
from ipywidgets import interact_manual

@interact_manual
def get_joint_value():
    jntPos = [0.] * 6
    ret = nmc.groupGetActualPosAcs(jntPos)
    if ret != SUCCESS:
        print( "NMC_GroupGetActualPosAcs error, error code =", ret)
    else:
        print( "NMC_GroupGetActualPosAcs:")
        for i in range(0, 6, 1):
            print( "   Joint", i ,"val:", '%.3f' % jntPos[i])
        nmv.update_tf_view(jntPos)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

## 11. Execute Group PTP motion to a pre-define position
##### Note: Please make sure all the joints are finish homing (if controlling real MiniBOT)

In [13]:
desPos = [0, 90, 0, 0, -90, 0]
nmc.groupPtpAcsAll(desPos)

0

#### 12. Show Actual Cartesian Position 
This step can be called to check actual cartesian values.

In [16]:
#get current cartesian value
cartPos = [0.] * 6
ret = nmc.groupGetActualPosPcs(cartPos)
if ret != SUCCESS:
    print( "groupGetActualPosPcs error, error code =", ret)
else:
    print( "groupGetActualPosPcs:")
    coord_list = ['x', 'y', 'z', 'a', 'b', 'c']
    for i in range(0, 6, 1):
        print( "\t", coord_list[i] ,"val:", '%.3f' % cartPos[i])

groupGetActualPosPcs:
	 x val: 300.000
	 y val: -99.992
	 z val: 414.005
	 a val: -0.002
	 b val: 0.001
	 c val: -180.000


## 13. Execute Group Line motion to a pre-define position
##### `Note: Please make sure all the joints are finish homing`

In [15]:
pos = [0.] * 6
nmc.groupGetActualPosPcs(pos)
pos[0] += 50; pos[1] += -100; pos[2] += -150
nmc.groupLine(pos)

0

### 15. record current pose (Joint space and Cartesian space)

In [17]:
nmc.recordPoint()

0

### Save and Read Point from csv file

In [18]:
nmc.savePoint('test_pnt.csv')

In [19]:
# Note: this will remove prvious record point list.
nmc.readPoint('record_pnt.csv')

#### 16. show recorded points

In [20]:
from IPython.display import display, HTML

df = pd.DataFrame(data=nmc.pnt_list, 
                  columns=['j1','j2','j3','j4','j5','j6','x','y','z','a','b','c'])

display( HTML(df.to_html()) )

,j1,j2,j3,j4,j5,j6,x,y,z,a,b,c
0,0.002045,53.142955,0.002045,2.700000e-03,-51.925500,6.750000e-03,394.001421,0.017590,350.118086,-0.006372,-1.219500,-179.997695
1,0.000682,90.000000,0.000682,9.000000e-04,-90.000000,-8.673617e-19,250.000297,0.004467,564.002975,0.000682,-0.000682,-179.999100
2,14.689773,97.049318,-26.008636,2.168404e-19,-71.041500,1.468800e+01,221.028975,57.943699,477.088505,0.001773,0.000791,179.999793
3,-16.078636,82.436591,-46.300227,2.168404e-19,-36.137250,-1.607850e+01,227.599172,-65.601201,331.200639,-0.000136,0.000852,-179.999755
4,-38.658409,55.913864,-27.647727,2.168404e-19,-28.265625,-3.865725e+01,250.004292,-199.993421,264.009383,-0.001159,-0.000399,179.999681


## 17. move PTP similar to GRC
Note: movePTP is blocking call function. The `Run Interact` button will not response before move command finished.   Please wait the cell from `[*]`(busy) to `[number]`(finsih)

In [21]:
nmc.movePTP(1)

0

## 18 move Line similar to GRC
Note: Same as movePTP, moveLine is blocking call function.

In [22]:
nmc.moveLine(4)

0

## 19. Halt Robot Movement
Note: `goupHalt` will not work on `movePTP` and `moveLine`. It can work on buit-in contorl command API, such as `groupPtpAcsAll` and `geoupLine`.

In [23]:
nmc.groupHalt()

0

## 20. Set DO value
Use Digital Output to control gripper or suctions. For simulation, this one can skip.

In [24]:
DO_list = [[0,2,4],[]]  # Turn digital oupt 0, 2, 4 to `on` and other to `off`
nmc.writeOutputMemory(DO_list)

0

### Write Script
Note: `Run Interact` button and `goupHalt` function is not work in this script (Because it use `movePTP` and `moveLine`). For simulation, this one can skip. If run this cell, it will take a minute to finish. While the below cell busy `[*]`, you can not run other cells.

In [24]:
import time
nmc.movePTP(0)
nmc.movePTP(1)
nmc.movePTP(2)
nmc.moveLine(3)
nmc.writeOutputMemory([[2],[]])
for i in range(2):
    nmc.moveLine(4)
    nmc.moveLine(3)
nmc.writeOutputMemory([[],[]])
nmc.moveLine(2)
nmc.moveLine(1)
nmc.movePTP(0)


0

### Compute Base transformation pose

In [25]:
coordTrans = [0.]*6
nmc.baseCalib_2p(nmc.pnt_list[0][6:], nmc.pnt_list[1][6:], coordTrans)
print(coordTrans)

[394.001421234, 0.0175899736321, 350.118085994, -179.99477867616073, -0.0, 0.0]


### Get Transformation matrix

In [26]:
TrMat = nm.pose2matrix(coordTrans)
print(np.around(TrMat, decimals=2))

[[-1.0000e+00  0.0000e+00  0.0000e+00  3.9400e+02]
 [-0.0000e+00 -1.0000e+00  0.0000e+00  2.0000e-02]
 [ 0.0000e+00  0.0000e+00  1.0000e+00  3.5012e+02]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00]]


In [27]:
# From robot-base position transform to pixel-based position
pos = [0.] * 6
nmc.groupGetActualPosPcs(pos)
print(pos)
TrInv = np.linalg.inv(TrMat)
tcpMat = nm.pose2matrix(pos) # [340, 200, 200, 0, 0, 180]
print(np.around(TrInv.dot(tcpMat), decimals=1))
print(np.around(TrInv.dot(np.mat([340, 150, 200, 1]).transpose()), decimals=1))

# From pixel-base position transform to robot-base position
pixMat = nm.pose2matrix([50, 0, 0, 0, 0, 0])
print(np.around(TrMat.dot(pixMat), decimals=2))
print(np.around(TrMat.dot(np.mat([250, 0, 0, 1]).transpose()), decimals=2))

[250.00236799235495, -199.99188229288225, 264.0110500678096, -0.0011590897949959038, -0.0003993221782678587, 179.9996805714971]
[[ -1.    0.   -0.  144. ]
 [  0.    1.    0.  200. ]
 [  0.    0.   -1.  -86.1]
 [  0.    0.    0.    1. ]]
[[  54. ]
 [-150. ]
 [-150.1]
 [   1. ]]
[[-1.0000e+00  0.0000e+00  0.0000e+00  3.4400e+02]
 [-0.0000e+00 -1.0000e+00  0.0000e+00  1.0000e-02]
 [ 0.0000e+00  0.0000e+00  1.0000e+00  3.5012e+02]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00]]
[[ 1.4400e+02]
 [-1.0000e-02]
 [ 3.5012e+02]
 [ 1.0000e+00]]


### 21. Disable all axes of group (Servo Off)
MiniBot will make a sound when it is disabled.

In [28]:
nmc.groupDisable()

0

### 22. Shutdown device

In [29]:
nmc.deviceShutdown()

0